In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn")
from scipy.optimize import minimize

In [ ]:
import SMA
import BollingerBands

In [ ]:
def optimal_strategy(parameters):
    start = "2011-01-03 00:00:00"
    end = "2021-5-31 20:00:00"
    symbol = "USDINR"
    tc = 0.00005
    
    # SMA
    tester1 = SMA.SMABacktester(symbol, int(parameters[0]), int(parameters[1]), start, end, tc)
    tester1.test_strategy()
    
    # Bollinger
    tester2 = BollingerBands.BBBacktester(symbol,  int(parameters[2]),  int(parameters[3]), start, end, tc)
    tester2.test_strategy()
    
    # Create comb
    comb = tester1.results.loc[:, ["returns", "position"]].copy()
    comb.rename(columns = {"position":"position_SMA"}, inplace = True)
    comb["position_MR"] = tester2.results.position
    
    comb["position_comb"] = np.sign(comb.position_MR + comb.position_SMA)
    
    comb["hour"] = comb.index.hour
    comb.position_comb = np.where(comb.hour.between(3.30, 11.30), comb.position_comb, 0)
    
    # Backtest
    comb["strategy"] = comb["position_comb"].shift(1) * comb["returns"]
    comb.dropna(inplace=True)
    comb["trades"] = comb.position_comb.diff().fillna(0).abs()
    comb.strategy = comb.strategy - comb.trades * tc
    comb["creturns"] = comb["returns"].cumsum().apply(np.exp)
    comb["cstrategy"] = comb["strategy"].cumsum().apply(np.exp)
    
    return -comb["cstrategy"].iloc[-1]

# opts = minimize(optimal_strategy, start_par, method = "Powell" , bounds = bnds)
# 55, 160, 59, 3

In [ ]:
optimal_strategy((46, 81, 9, 3))

In [ ]:
bnds =  ((30, 60), (70, 100), (5, 50), (1, 5))

In [ ]:
start_par = (46, 81, 9, 3)

In [ ]:
opts = minimize(optimal_strategy, start_par, method = "Powell" , bounds = bnds)

In [ ]:
opts

In [ ]:
optimal_strategy(opts.x)